In [3]:
 #importing required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#import library for metrics
from sklearn import metrics
#import libraries for ROC
import matplotlib.pyplot as plt

Section 1:

In [43]:
#reading and storing dataset into dataframe
cancer_df  = pd.read_csv("https://github.com/mpourhoma/CS4662/raw/master/Cancer.csv")
cancer_df

,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Malignant_Cancer
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...
145,3,1,1,1,2,1,2,1,1,0
146,9,7,7,5,5,10,7,8,3,1
147,10,8,8,4,10,10,8,1,1,1
148,1,1,1,1,2,1,3,1,1,0


In [79]:
# create a python list of feature names that would like to pick from the dataset:
feature = ['Clump_Thickness','Uniformity_of_Cell_Size','Uniformity_of_Cell_Shape',
                'Marginal_Adhesion','Single_Epithelial_Cell_Size','Bare_Nuclei',
                'Bland_Chromatin','Normal_Nucleoli','Mitoses']

# use the above list to select the features from the original DataFrame
X = cancer_df[feature] 

# select a Series of labels (the last column) from the DataFrame
y = cancer_df['Malignant_Cancer']

Section 2:

In [80]:
# Randomly splitting the original dataset into training set and testing set:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)
#import DT classifier
from sklearn.tree import DecisionTreeClassifier
#"my_DecisionTree" is instantiated as an "object" of DecisionTreeClassifier "class".
my_DecisionTree = DecisionTreeClassifier(random_state=2)

Section 3:

In [63]:
#Training Decision Tree
my_DecisionTree.fit(X_train, y_train)
#Test my_DecisionTree on testing set
my_DecisionTree_predict = my_DecisionTree.predict(X_test)

# Estimating the probability (likelihood) of Each Label: 
my_DecisionTree_predict_prob = my_DecisionTree.predict_proba(X_test)

#import accuracy library
from sklearn.metrics import accuracy_score
#finding accuracy of our model
my_DecisionTree_accuracy = accuracy_score(y_test, my_DecisionTree_predict)
#print the accuracy
print("DT accuracy ",my_DecisionTree_accuracy)

DT accuracy  0.8666666666666667


In [64]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, my_DecisionTree_predict_prob[:,1], pos_label=1)
# AUC:
AUC_DT = metrics.auc(fpr, tpr)
print("DT AUC ",AUC_DT)

DT AUC  0.8697478991596639


Section 4: Bagging

In [81]:
#import resample library
from sklearn.utils import resample

bootstarp_size = int(0.8 * X_train.shape[0])
#create empty list
BDTsamples = []

for i in range(29):
    X,y = resample(X_train, y_train, n_samples = bootstarp_size, random_state = i, replace = True)
    Base_DecisionTree = DecisionTreeClassifier(random_state = 2)
    Base_DecisionTree.fit(X,y)
    #testing Base_DecisionTree on orginial testing set
    Base_DecisionTree_predict = Base_DecisionTree.predict(X_test)
    #storing predicitons in list
    BDTsamples.append(Base_DecisionTree_predict)
    
#printing prediction results from Base_DecisionTree tested on originial testing set
#votes of 29 classifiers
#for i in BDTsamples:
#    print(i)

In [82]:
#creating empty list to for voting
majority = []

Base_Tree = pd.DataFrame(BDTsamples).T
column_list = list(Base_Tree)
Base_Tree["sum"] = Base_Tree[column_list].sum(axis=1).T
sum_list = Base_Tree["sum"].T.tolist()
#print(Base_Tree["sum"])

for i in sum_list:
    if i > (int(len(BDTsamples[0])/2)):
        majority.append(1)
    else:
        majority.append(0)

#print(majority)        

#calculate accuracy score of majority list
majority_DT_accuracy = accuracy_score(y_test, majority)
print("Voting accuracy: ", majority_DT_accuracy)

Voting accuracy:  0.8222222222222222


In [75]:
#create empty list for average of votes
avgPredictions = []
#iterate through list of row sums and divide by length of list for average
for i in sum_list:
    avg = i / len(Base_Tree[0])
    avgPredictions.append(avg)
    
#print(avgPredictions)

fpr2, tpr2, thresholds2 = metrics.roc_curve(y_test, avgPredictions, pos_label=1)

voting_AUC = metrics.auc(fpr2,tpr2)
print("\nVoting AUC: ", voting_AUC)


Voting AUC:  0.9380252100840336


Section 5: AdaBoost

In [76]:
#import AdaBoost
from sklearn.ensemble import AdaBoostClassifier
my_AdaBoost = AdaBoostClassifier(n_estimators=29,random_state=2)
my_AdaBoost.fit(X_train,y_train)
my_AdaBoost_predict = my_AdaBoost.predict(X_test)
my_AdaBoost_predict_prob = my_AdaBoost.predict_proba(X_test)
AdaBoost_accuracy = accuracy_score(y_test, my_AdaBoost_predict)
print("AdaBoost accuracy: ", AdaBoost_accuracy)

fpr3, tpr3, thresholds3 = metrics.roc_curve(y_test, my_AdaBoost_predict_prob[:,1], pos_label=1)
ada_AUC = metrics.auc(fpr3, tpr3)
print("\nAdaBoost AUC: ", ada_AUC)

AdaBoost accuracy:  0.9555555555555556

AdaBoost AUC:  0.9653361344537815


Section 6: XGBoost


#pip install xgboost


#brew install libomp


In [72]:
#use_label_encoder was suggested to stop the bright red error message
from xgboost import XGBClassifier
my_XGBoost = XGBClassifier(n_estimators=29,random_state=2,use_label_encoder=False)
my_XGBoost.fit(X_train,y_train)
my_XGBoost_predict = my_XGBoost.predict(X_test)
my_XGBoost_predict_prob = my_XGBoost.predict_proba(X_test)
XBG_accuracy = accuracy_score(y_test, my_XGBoost_predict)
print("XGB accuracy: ",XBG_accuracy)

fpr4, tpr4, thresholds4 = metrics.roc_curve(y_test, my_XGBoost_predict_prob[:,1],pos_label=1)
XGB_AUC = metrics.auc(fpr4,tpr4)
print("\nXGB AUC: ", XGB_AUC)

[00:10:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGB accuracy:  0.9555555555555556

XGB AUC:  0.9716386554621849


Section 7: Random Forest

In [73]:
from sklearn.ensemble import RandomForestClassifier
my_RandomForest = RandomForestClassifier(n_estimators=29,bootstrap = True, random_state=2)
my_RandomForest.fit(X_train,y_train)
my_RandomForest_predict = my_RandomForest.predict(X_test)
my_RandomForest_predict_prob = my_RandomForest.predict_proba(X_test)
my_RandomForest_accuracy = accuracy_score(y_test, my_RandomForest_predict)
print("Random Forest accuracy: ", my_RandomForest_accuracy)

fpr5,tpr5,thresholds5 = metrics.roc_curve(y_test, my_RandomForest_predict_prob[:,1],pos_label=1)
RF_AUC = metrics.auc(fpr5,tpr5)
print("\nRandom Forest AUC: ", RF_AUC)

Random Forest accuracy:  0.9555555555555556

Random Forest AUC:  0.9516806722689075


Section 8: ROC Curve

In [83]:
#keep figures inside jupyter notebook
%matplotlib inline

plt.figure()

#ROC curve
plt.plot(fpr, tpr, color = 'red', lw = 2, label = 'Decision Tree (area = %0.2f)' % AUC_DT)
plt.plot(fpr2, tpr2, color = 'green', lw = 2, label = 'Voting  (area = %0.2f)' % voting_AUC)
plt.plot(fpr3, tpr3, color = 'blue', lw = 2, label = 'AdaBoost (area = %0.2f)' % ada_AUC)
plt.plot(fpr4, tpr4, color = 'purple', lw = 2, label = ' XGBoost (area = %0.2f)' % XGB_AUC)
plt.plot(fpr5, tpr5, color = 'orange', lw = 2, label = 'RandomForest (area = %0.2f)' % RF_AUC)

#Random Guess Line:
plt.plot([0, 1], [0, 1], color = 'cyan', lw = 1, linestyle = '--')

#Defining The Range of X-Axis and Y-Axis
plt.xlim([-0.005, 1.005])
plt.ylim([0.0, 1.01])

#Labels, Title, Legend:
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Cancer Predicter')
plt.legend(loc = "lower right")

plt.show()

ValueError: Invalid RGBA argument: ''

<Figure size 432x288 with 1 Axes>

In [59]:
#Accuracies
print("DT accuracy ",my_DecisionTree_accuracy)
print("Voting accuracy: ", majority_DT_accuracy)
print("AdaBoost accuracy: ", AdaBoost_accuracy)
print("XGB accuracy: ",XBG_accuracy)
print("Random Forest accuracy: ", my_RandomForest_accuracy)

DT accuracy  0.8666666666666667
Voting accuracy:  0.8222222222222222
AdaBoost accuracy:  0.9555555555555556
XGB accuracy:  0.9555555555555556
Random Forest accuracy:  0.9555555555555556


In [60]:
#AUC
print("DT AUC ",AUC_DT)
print("Voting AUC: ", voting_AUC)
print("AdaBoost Auc: ", ada_AUC)
print("XGB AUC: ", XGB_AUC)
print("Random Forest AUC: ", RF_AUC)

DT AUC  0.8697478991596639
Voting AUC:  0.9380252100840336
AdaBoost Auc:  0.9653361344537815
XGB AUC:  0.9716386554621849
Random Forest AUC:  0.9516806722689075


Which algorithm is the best w.r.t the AUC value? --
XGBoost

Which algorithm is the best w.r.t the Accuracy value? --
AdaBoost, XGBoost, and RandomForest are tied

Which algorithm is the best when we want a False Positive Rate of %7? --
Random Forest